In [87]:
import pandas as pd

from shapely.geometry import Point, Polygon

import geopandas as gp

import numpy as np

import random

import pulp

import folium

import networkx as nx

import math

import h3

import copy

import sys

import time

'''User defined functions'''
def visualize_grids(grids,folium_map):
    
    for grid in grids:
        boundary=h3.h3_to_geo_boundary(grid)
        boundary=[x for x in boundary]
        boundary.append(boundary[0])
        grid_PolyLine=folium.PolyLine(locations=boundary,weight=4,color='red')
        folium_map.add_child(grid_PolyLine) 
    return folium_map

'''Get Neighbor ranges'''

def Get_neighbors(y):
    x=list()
    for y_ in y:
        for y__ in y_:
            x.append(y__)
    return x

'''Compact ranges'''

def Compact_lists(arr):
    
    result=list()
    
    for a in arr:
        
        result=list(set(result+a))
        
    return result

'''Get Broadcast Capacity'''

def Get_q(fee,q):
    
    fee=min(100,fee)
    
    fee=max(40,fee)
    
    fee=int(fee/10)
    
    return q[fee]


'''Optimizaton'''

def MILP_Optimization(Weight,Capacity_O,Capacity_D):

    '''Define the problem'''

    model = pulp.LpProblem("Matching", pulp.LpMaximize)

    '''Construct our decision variable lists'''

    X = pulp.LpVariable.dicts("X",((D,O) for D in Weight.keys() for O in Weight[D].keys() if O!=-1),lowBound=0,upBound=1,cat='Integer')

    '''Objective Function'''

    model += (pulp.lpSum([Weight[D][O] * X[(D, O)] for D in Weight.keys() for O in Weight[D].keys() if O!=-1]))


    '''Browse  constraints'''

    for D in Weight.keys():
        
        model += pulp.lpSum([X[(D, O)] for O in Weight[D].keys() if O!=-1]) <= Capacity_D[D]

    '''Broadcast constraints'''
    Weight_={}
    for D in Weight.keys():
        for O in Weight[D].keys():
            if O not in Weight_.keys():
                Weight_[O]={}
                Weight_[O][D]=Weight[D][O]
            else:
                Weight_[O][D]=Weight[D][O]

    for O in Weight_.keys():
        if O!=-1:
            model += pulp.lpSum([X[(D, O)] for D in Weight_[O].keys()]) <= Capacity_O[O]


    model.solve()

    result={}

    for var in X:

        var_value = X[var].varValue
        if var_value !=0:
            if var[0] not in result.keys():
                result[var[0]]=[var[1]]
            else:
                result[var[0]].append(var[1])

    return result

def Reposition(lat,lng,grid,Order_count,speed):
    
    loc=Point(lat,lng)
    
    neighbors= Get_neighbors(h3.hex_range_distances(grid, 10))
    candidates=[g for g in Order_count.keys() if Order_count[g]>0]
    Intersects=[g for g in neighbors if g in candidates]

    if len(Intersects)==0:
        r_grid=random.choice(neighbors)
    else:
        score={g:Order_count[g] for g in Intersects}
        r_grid=max(score, key=score.get)
        
    r_lat,r_lng=h3.h3_to_geo(r_grid)[0],h3.h3_to_geo(r_grid)[1]
    
    r_dest=Point(r_lat,r_lng)
    
    if lat!=r_lat and lng!=r_lng:

        dis=r_dest.distance(loc)*111*1.1

        ratio=speed/dis

        if ratio<=1:

            pnt_lng=lng+(r_lng-lng)*ratio

            pnt_lat=lat+(r_lat-lat)*ratio

        else:

            pnt_lng=r_lng

            pnt_lat=r_lat
            
    else:
        
        pnt_lng=lng

        pnt_lat=lat
        
        
    return pnt_lat,pnt_lng

In [88]:
'''Param'''

Strategy='OPT'

Start_step=2520

End_step=3600

Max_waiting=60

'''Broadcasting radius'''

radius=10

speed=0.1 # 10 m/second= 0.1 km/10second

alpha=0.15

beta=2

gamma=0.5

lambda_=0.8

u_0=25

w=10

Q={4:300,5:200,6:150,7:100,8:50,9:20,10:10}

Grids=np.load('./Data/Processed/Grids.npy',allow_pickle=True)


In [89]:


Order_df=pd.read_csv('./Data/Processed/order_table.csv')

Order_df=Order_df.drop(columns=['Unnamed: 0'])

Order_df=Order_df.drop_duplicates(subset=['order_id'])

Order_df['driver_id']='Waiting'

Order_df['response_step']=End_step

Order_df['pickup_step']=End_step

Order_df=Order_df[['order_id',\
                   'driver_id',
                   'create_step',\
                   'response_step',\
                   'pickup_step',\
                   'start_lng',\
                   'start_lat',\
                   'end_lng',\
                   'end_lat',\
                   'fee',\
                   'start_grid',\
                   'end_grid']]

Driver_df=pd.read_csv('./Data/Processed/driver_table.csv')

Driver_df['order_id']='Idle'

Driver_df=Driver_df.drop(columns=['Unnamed: 0'])

Driver_df=Driver_df[['driver_id',\
                   'order_id',
                   'step',\
                   'longitude',\
                   'latitude',\
                   'grid']]

Driver_num=Driver_df.shape[0]

Quatitive_results=list()

Broadcast_item={'order_id': list(),\
                'driver_id':list(),\
                'origin':list(),\
                'loc':list(),\
                'step':list(),\
                'fee':list(),\
                'utility':list(),\
                'response':list()}

Grid_range=list(set(Order_df['start_grid']))

Cancel_num=0

Matched_num=0


In [90]:
for step in range(Start_step,End_step,1):


    '''(1) select unserved orders and arriving orders'''

    Order_batch=Order_df[(Order_df['create_step']<=step)&(Order_df['driver_id']=='Waiting')]

    Order_info={}

    for idx,row in Order_batch.iterrows():

        order_id=int(row['order_id'])

        origin=Point(row['start_lng'],row['start_lat'])

        dest=Point(row['end_lng'],row['end_lat'])

        Order_info[order_id]={'origin':origin,\
                              'dest':dest,\
                              'origin_grid':row['start_grid'],\
                              'dest_grid':row['end_grid'],\
                              'start_lng':row['start_lng'],\
                              'start_lat':row['start_lat'],\
                              'end_lng':row['end_lng'],\
                              'end_lat':row['end_lat'],\
                              'Travel_time':int((origin.distance(dest)*111*1.1)/speed),\
                              'fee':row['fee'],\
                              'broadcast_limit':Get_q(row['fee'],Q),\
                              'match_grids':Get_neighbors(h3.hex_range_distances(row['start_grid'], radius))}

    Operation_grids=Compact_lists([x['match_grids'] for x in Order_info.values()])


    '''(2) select Idle drivers'''

    Idle_drivers=list(Driver_df.loc[(Driver_df['step']==step)&(Driver_df['order_id']=='Idle'),'driver_id'])

    Driver_batch=Driver_df[(Driver_df['step']==step)&(Driver_df['order_id']=='Idle')&(Driver_df['grid'].isin(Operation_grids))]

    Driver_info={}

    for idx,row in Driver_batch.iterrows():

        driver_id=int(row['driver_id'])

        loc=Point(row['longitude'],row['latitude'])

        Driver_info[driver_id]={'grid':row['grid'],\
                                'loc':loc,\
                                'browse_limit':w}

    '''(3) Utility Matrix'''

    Utility={driver_id:{'O1':{},'O0':{-1:u_0}} for driver_id in Driver_info.keys()}

    Pickup_distance={driver_id:{} for driver_id in Driver_info.keys()}

    Pickup_time={driver_id:{} for driver_id in Driver_info.keys()}

    Reward={driver_id:{} for driver_id in Driver_info.keys()}

    Scaling=lambda x,alpha:1-np.exp(-1*alpha*x)

    for driver_id in Driver_info.keys():

        grid=Driver_info[driver_id]['grid']; loc=Driver_info[driver_id]['loc']

        for order_id in Order_info.keys():

            if grid in Order_info[order_id]['match_grids']:

                origin=Order_info[order_id]['origin']

                Pickup_distance[driver_id][order_id]=origin.distance(loc)*111*1.1

                Reward[driver_id][order_id]=1/Pickup_distance[driver_id][order_id]

                Pickup_time[driver_id][order_id]=max(int(Pickup_distance[driver_id][order_id]/speed),1)

                Utility[driver_id]['O1'][order_id]=Order_info[order_id]['fee']-Pickup_distance[driver_id][order_id]*beta

        Mean_Utility=np.array(list(Utility[driver_id]['O1'].values())).mean()

        Deviation=Mean_Utility-u_0

        Quantity=len(Utility[driver_id]['O1'])

        Utility[driver_id]['O0'][-1]=u_0+Deviation*Scaling(Quantity,alpha)

    '''(4) Nested Multinomial Logit Model'''

    V_value = {driver_id:{'O1':0,'O0':0} for driver_id in Driver_info.keys()}

    P_nest = {driver_id:{'O1':0,'O0':0} for driver_id in Driver_info.keys()}

    P_item = {driver_id:{'O1':{order_id:0 for order_id in Utility[driver_id]['O1'].keys()},'O0':{-1:1}} for driver_id in Driver_info.keys()}

    Prob = {driver_id:{order_id:0 for order_id in list(Utility[driver_id]['O1'].keys())+[-1]} for driver_id in Driver_info.keys()}

    for driver_id in Driver_info.keys():

        # Nest-choosing probability

        V_value[driver_id]['O1']=np.log(sum([np.exp(gamma*u) for u in Utility[driver_id]['O1'].values()]))

        V_value[driver_id]['O0']=gamma*Utility[driver_id]['O0'][-1]

        if np.exp(lambda_*V_value[driver_id]['O1']) == float('inf'):

            P_nest[driver_id]['O1']=1.0; P_nest[driver_id]['O0']=0.0;

        elif np.exp(lambda_*V_value[driver_id]['O0']) == float('inf'):

            P_nest[driver_id]['O1']=0.0; P_nest[driver_id]['O0']=1.0;

        else:

            P_nest[driver_id]['O1']=np.exp(lambda_*V_value[driver_id]['O1'])/(np.exp(lambda_*V_value[driver_id]['O1'])+np.exp(lambda_*V_value[driver_id]['O0']))

            P_nest[driver_id]['O0']=np.exp(lambda_*V_value[driver_id]['O0'])/(np.exp(lambda_*V_value[driver_id]['O1'])+np.exp(lambda_*V_value[driver_id]['O0']))

        # Nest-based item-choosing probability

        denominator=sum([np.exp(gamma*u) for u in Utility[driver_id]['O1'].values()])

        if denominator!=float('inf'):

            for order_id in Utility[driver_id]['O1'].keys():

                P_item[driver_id]['O1'][order_id]=np.exp(gamma*Utility[driver_id]['O1'][order_id])/denominator

        else:

            for order_id in Utility[driver_id]['O1'].keys():

                if np.exp(Utility[driver_id]['O1'][order_id]) == float('inf'):

                    P_item[driver_id]['O1'][order_id]=1.0

                    break 

        # Item-choosing probability

        for order_id in P_item[driver_id]['O1'].keys():

            Prob[driver_id][order_id]=P_nest[driver_id]['O1']*P_item[driver_id]['O1'][order_id]

        Prob[driver_id][-1]=P_nest[driver_id]['O0']*P_item[driver_id]['O0'][-1]

    '''(5) Optimization'''

    if Strategy =='OPT':

        Capacity_O={O:Order_info[O]['broadcast_limit'] for O in Order_info.keys()}

        Capacity_D={D:Driver_info[D]['browse_limit'] for D in Driver_info.keys()}

        Weight=Prob

        Displayed_Solution=MILP_Optimization(Weight,Capacity_O,Capacity_D)

    '''(6) Display strategy'''

    # Baseline

    if Strategy !='OPT':

        Dispaly_Matrix=Prob

    else:

        print('Optimization!')

        print('+'*50)

        Dispaly_Matrix={driver_id:{order_id:Prob[driver_id][order_id] for order_id in Displayed_Solution[driver_id]+[-1]} for driver_id in Displayed_Solution.keys()}

        for driver_id,displayed in Dispaly_Matrix.items():

            denominator=sum(displayed.values())

            for order_id in displayed.keys():

                Dispaly_Matrix[driver_id][order_id]=Dispaly_Matrix[driver_id][order_id]/denominator

    # Record

    for driver_id in Dispaly_Matrix.keys():

        for order_id in Dispaly_Matrix[driver_id]:

            if order_id !=-1:

                Broadcast_item['order_id'].append(order_id);

                Broadcast_item['driver_id'].append(driver_id);

                Broadcast_item['origin'].append(Order_info[order_id]['origin']);

                Broadcast_item['loc'].append(Driver_info[driver_id]['loc']);

                Broadcast_item['step'].append(step);

                Broadcast_item['fee'].append(Order_info[order_id]['fee']);

                Broadcast_item['utility'].append(Utility[driver_id]['O1'][order_id]);

                Broadcast_item['response'].append(0);


    '''(7) Simulate drivers' choice'''

    Choice={}

    for driver_id in Dispaly_Matrix.keys():

        order_id=np.random.choice(list(Dispaly_Matrix[driver_id].keys()), p=list(Dispaly_Matrix[driver_id].values()))

        Choice[driver_id]=order_id

        if  order_id!=-1:

            Broadcast_item['order_id'].append(order_id);

            Broadcast_item['driver_id'].append(driver_id);

            Broadcast_item['origin'].append(Order_info[order_id]['origin']);

            Broadcast_item['loc'].append(Driver_info[driver_id]['loc']);

            Broadcast_item['step'].append(step);

            Broadcast_item['fee'].append(Order_info[Choice[driver_id]]['fee']);

            Broadcast_item['utility'].append(Utility[driver_id]['O1'][order_id]);

            Broadcast_item['response'].append(1);

    Responding_num=len([driver_id for driver_id,order_id in Choice.items() if order_id!=-1])

    '''(8) Ranking and Assigning'''

    Assign={}

    Matching_result={}

    for driver_id,order_id in Choice.items():

        if order_id not in Assign.keys():

            Assign[order_id]=[driver_id]

        else:

            Assign[order_id].append(driver_id)

    for order_id in Assign.keys():

        if order_id!=-1:

            Rank_matrix={driver_id:Reward[driver_id][order_id] for driver_id in Assign[order_id]}

            driver_id=max(Rank_matrix, key=Rank_matrix.get)

            Matching_result[order_id]=driver_id

    Matched_num+=len(Matching_result)   

    '''(9) Update results of orders'''

    '''(9-1) Update the matched orders and drivers'''

    Added_item={'driver_id': list(),\
                'order_id':list(),\
                'step':list(),\
                'longitude':list(),\
                'latitude':list(),\
                'grid':list()}

    for order_id,driver_id in Matching_result.items():

        '''Matched order'''

        Order_df.loc[(Order_df['create_step']<=step)&(Order_df['order_id']==order_id),'response_step']=step

        Order_df.loc[(Order_df['create_step']<=step)&(Order_df['order_id']==order_id),'pickup_step']=step+Pickup_time[driver_id][order_id]

        Order_df.loc[(Order_df['create_step']<=step)&(Order_df['order_id']==order_id),'driver_id']=driver_id

        '''Matched driver'''

        Driver_df.loc[(Driver_df['step']==step)&(Driver_df['driver_id']==driver_id)&(Driver_df['grid'].isin(Operation_grids)),'order_id']=order_id

        Added_item['driver_id'].append(driver_id);

        Added_item['order_id'].append('Idle');

        Added_item['step'].append(step+Pickup_time[driver_id][order_id]+Order_info[order_id]['Travel_time']);

        Added_item['longitude'].append(Order_info[order_id]['end_lng']);

        Added_item['latitude'].append(Order_info[order_id]['end_lat']);

        Added_item['grid'].append(Order_info[order_id]['dest_grid']);


    Added_df=pd.DataFrame(data=Added_item)

    Driver_df=pd.concat([Driver_df,Added_df],ignore_index=True)

    '''(9-2) Update the unmatched orders'''

    Unmatched_orders=[O for O in Order_info.keys() if O not in Matching_result.keys()]

    Order_count={g:0 for g in Grid_range}

    for order_id in Unmatched_orders:

        grid=Order_info[order_id]['origin_grid']

        Order_count[grid]+=1

    if len(Unmatched_orders)!=0:

        Cancel_num+=Order_df.loc[((step-Order_df['create_step'])>Max_waiting)&(Order_df['order_id'].isin(Unmatched_orders)),'driver_id'].shape[0]

        Order_df.loc[((step-Order_df['create_step'])>Max_waiting)&(Order_df['order_id'].isin(Unmatched_orders)),'driver_id']='Canceled'


    '''(9-3) Update the unmatehed drivers: Repositioning'''

    Idle_drivers=list(Driver_df.loc[(Driver_df['step']==step)&(Driver_df['order_id']=='Idle'),'driver_id'])

    if len(Idle_drivers)!=0:

        Next_Driver_df=copy.copy(Driver_df.loc[(Driver_df['step']==step)&(Driver_df['driver_id'].isin(Idle_drivers))])

        '''Local hot'''

        Next_Driver_df['step']=Next_Driver_df.apply(lambda x:x['step']+1,axis=1)

        Next_Driver_df['Tuple']=Next_Driver_df.apply(lambda x:Reposition(x['latitude'], x['longitude'], x['grid'],Order_count,speed),axis=1)

        Next_Driver_df['latitude']=Next_Driver_df.apply(lambda x:x['Tuple'][0],axis=1)

        Next_Driver_df['longitude']=Next_Driver_df.apply(lambda x:x['Tuple'][1],axis=1)

        Next_Driver_df['grid']=Next_Driver_df.apply(lambda x:h3.geo_to_h3(x['latitude'], x['longitude'], 8),axis=1)

        Next_Driver_df=Next_Driver_df[['driver_id',\
                           'order_id',
                           'step',\
                           'longitude',\
                           'latitude',\
                           'grid']]

        Driver_df=pd.concat([Driver_df,Next_Driver_df],ignore_index=True)

    '''(10) Summary'''

    if len(Order_info)!=0:

        responding_rate=len(Matching_result)/len(Order_info)

    else:

        responding_rate=-1.0

    print('*'*50)

    print('Current step: ', step)

    print('Cumulative Matched pairs: ',Matched_num,'Cumulative Canceled orders: ',Cancel_num)

    print('Real-time Matched pairs: ', len(Matching_result),'Real-time responding drivers: ',Responding_num)

    print('Real-time responding rate: ', responding_rate,'Real-time responding rate: ',Responding_num/Driver_num)

    Quatitive_results.append([step,Matched_num,Responding_num,responding_rate])

    break

Optimization!
++++++++++++++++++++++++++++++++++++++++++++++++++
**************************************************
Current step:  2520
Cumulative Matched pairs:  2 Cumulative Canceled orders:  0
Real-time Matched pairs:  2 Real-time responding drivers:  110
Real-time responding rate:  1.0 Real-time responding rate:  0.025937278943645368


In [95]:
Order_info

{439952024: {'origin': <shapely.geometry.point.Point at 0x7efc11a0ec90>,
  'dest': <shapely.geometry.point.Point at 0x7efc118da050>,
  'origin_grid': '88411caa45fffff',
  'dest_grid': '88411cae65fffff',
  'start_lng': 114.07083,
  'start_lat': 22.52918,
  'end_lng': 113.92228,
  'end_lat': 22.5268,
  'Travel_time': 181,
  'fee': 75,
  'broadcast_limit': 100,
  'match_grids': ['88411caa45fffff',
   '88411caa41fffff',
   '88411caa6bfffff',
   '88411caa4dfffff',
   '88411caa69fffff',
   '88411caa47fffff',
   '88411cb993fffff',
   '88411caa09fffff',
   '88411cb991fffff',
   '88411caa43fffff',
   '88411caa49fffff',
   '88411cb997fffff',
   '88411cb8a7fffff',
   '88411cb99bfffff',
   '88411caa6dfffff',
   '88411caa4bfffff',
   '88411caa63fffff',
   '88411caa61fffff',
   '88411caa0dfffff',
   '88411cb9bbfffff',
   '88411caa67fffff',
   '88411caa0bfffff',
   '88411cb8a3fffff',
   '88411cb999fffff',
   '88411cb995fffff',
   '88411cb8a5fffff',
   '88411cb99dfffff',
   '88411caa05fffff',
   '8841

In [94]:
Driver_df[Driver_df['driver_id']==2373958]



,driver_id,order_id,step,longitude,latitude,grid
1318,2373958,439952024,2520,114.07056,22.530413,88411caa6bfffff
4241,2373958,Idle,2702,113.92228,22.526800,88411cae65fffff
